In [207]:
import json
import sys
import os
import types
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [208]:
def getContentFromFile(filePath):
  return open(filePath, 'r').read()

In [209]:
def getContent(location):
  content = None
  if location.startswith("http"):
    content = getContentFromUri(location)
  else:
    content = getContentFromFile(location)
  if content is None:
    raise Error("Could not load content for " + location)
  return json.loads(content)

In [210]:
## same as what we have for json analyst
def extract_info_text(json1, entityList):
    for key_l1, value_l1 in json1.items():
        if key_l1 == "analyzeResult":
            for key_l2, value_l2 in value_l1.items():
                if key_l2 == "documentResults":
                    for i in range(len(value_l2)):
                        result = value_l2[i]
                        for key_l3, value_l3 in result.items():
                            if key_l3 == "fields":
                                result = []
                                for key_l4, value_l4 in value_l3.items():
#                                     print(key_l4)
                                    for item in entityList:
                                        
                                        tempList = []
                                        if key_l4 == item:
                                            tempList = [item, value_l4['text'], value_l4['boundingBox'][0], value_l4['boundingBox'][1], value_l4['boundingBox'][2], value_l4['boundingBox'][3], value_l4['boundingBox'][4], value_l4['boundingBox'][5], value_l4['boundingBox'][6], value_l4['boundingBox'][7] ]
                                            result.append(tempList)
                                                    
                                return result

In [265]:
## list the name of columns & targets and file names reading

invoiceDirectory = "/home/woody/桌面/2021Q6-main/Deloitte/dataset/" ## this location supposed to be where the result of code form_recignizer_invoice is
supportedExt = ['.json', '.jpg','.jpeg','.tif','.tiff','.png','.bmp']
columnList=['Y','value',"bb_loc_1", "bb_loc_2", "bb_loc_3", "bb_loc_4", "bb_loc_5", "bb_loc_6", "bb_loc_7", "bb_loc_8"]
entityList = ['CustomerAddress', 'VendorAddress', 'InvoiceId', 'PurchaseOrder']
fileNameList = []
for root, directories, filenames in os.walk(invoiceDirectory):
    for invoiceFilename in filenames:
        ext = os.path.splitext(invoiceFilename)[-1].lower()
        invoiceFilename = invoiceDirectory + invoiceFilename
        if ext in supportedExt:
            fileNameList.append(invoiceFilename)

In [266]:
## load data base on file names

raw_data_lists = []
for fileName in fileNameList:
    json_file = getContent(fileName)
    result = extract_info_text(json_file, entityList)
    raw_data_lists.append(result)

In [267]:
# transform data with only target info 

reshape_list = []
for raw_data_list in raw_data_lists:
    for item in raw_data_list:
        reshape_list.append(item)
df_final = pd.DataFrame(reshape_list, columns=columnList)
df_final.head()

,Y,value,bb_loc_1,bb_loc_2,bb_loc_3,bb_loc_4,bb_loc_5,bb_loc_6,bb_loc_7,bb_loc_8
0,CustomerAddress,ZEUGHOFSTRASSE 1 10997 BERLIN GERMANY,1.0697,1.7795,2.5096,1.7795,2.5096,2.1850,1.0697,2.1850
1,InvoiceId,855801065,6.4134,1.2319,7.0522,1.2319,7.0421,1.3435,6.4185,1.3435
2,VendorAddress,® Langer Kornweg 34K 65451 - Kelsterbach Germa...,1.3891,0.4411,3.0267,0.4411,3.0267,0.9176,1.3891,0.9176
3,CustomerAddress,Willy-Rüsch-Str. 4-10 71349 KERNEN,0.8883,2.2147,2.2033,2.2181,2.2021,2.6909,0.8871,2.6875
4,InvoiceId,80066508,7.0558,2.0709,7.7411,2.0709,7.7411,2.2029,7.0558,2.2029


In [268]:
## Bert function for word embedding 

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentences_List = []
for sentence in df_final.value:
    inputs = tokenizer(sentence)
    sentences_List.append(inputs['input_ids'])

In [269]:
# sentences_List
def find_max_list(list):
    list_len = [len(i) for i in list]
    return max(list_len)
print(find_max_list(sentences_List))

51


In [270]:
# set the length for embedding results
length_long_sentence = 60

In [271]:
# 0 padding for the result with length = 'length_long_sentence'
from keras.preprocessing.sequence import pad_sequences
padded_sentences = pad_sequences(sentences_List, length_long_sentence, padding='post')

In [272]:
# transpose the matrix for easier adding inside dataframe
padded_sentences = np.transpose(padded_sentences)

In [273]:
# name the columns
Columns_vectorized_list = ['column_01', 'column_02', 'column_03', 'column_04', 'column_05', 'column_06', 'column_07', 'column_08', 'column_09', 'column_10',
'column_11', 'column_12', 'column_13', 'column_14', 'column_15', 'column_16', 'column_17', 'column_18', 'column_19', 'column_20',
'column_21', 'column_22', 'column_23', 'column_24', 'column_25', 'column_26', 'column_27', 'column_28', 'column_29', 'column_30',
'column_31', 'column_32', 'column_33', 'column_34', 'column_35', 'column_36', 'column_37', 'column_38', 'column_39', 'column_40',
'column_41', 'column_42', 'column_43', 'column_44', 'column_45', 'column_46', 'column_47', 'column_48', 'column_49', 'column_50',
'column_51', 'column_52', 'column_53', 'column_54', 'column_55', 'column_56', 'column_57', 'column_58', 'column_59', 'column_60']

In [312]:
temp_list = padded_sentences.tolist()
# stand for the number of examples we have after transposation
print(len(temp_list[1]))

78


In [275]:
for i in range(len(temp_list)):
    df_final[Columns_vectorized_list[i]] = temp_list[i]

In [276]:
from flask import Flask, request, make_response
import pandas as pd
import numpy as np
import io
import re
import nltk
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [277]:
def preprocess(df, columnList):
    """Preprocess training dataframe.

    Parameters
    ----------
    df: pandas.DataFrame
        dataframe containing 'movie_id', 'synopsis' and 'genres' columns

    Returns
    -------
    df: pandas.DataFrame
        dataframe with labels als 0/1 columns
    """
    # df = df[columnList]
    for i in df.index:
        Ys = df.loc[i, 'Y'].split(' ')
        for Y in Ys:
            if not (Y in df.columns):
                df[Y] = 0
            df.loc[i, Y] = 1
    del df['Y']
    return df

In [278]:
df_final = preprocess(df_final, columnList)

In [279]:
labels = list(df_final.columns[69:])
df_final.head()

,value,bb_loc_1,bb_loc_2,bb_loc_3,bb_loc_4,bb_loc_5,bb_loc_6,bb_loc_7,bb_loc_8,column_01,...,column_55,column_56,column_57,column_58,column_59,column_60,CustomerAddress,InvoiceId,VendorAddress,PurchaseOrder
0,ZEUGHOFSTRASSE 1 10997 BERLIN GERMANY,1.0697,1.7795,2.5096,1.7795,2.5096,2.1850,1.0697,2.1850,101,...,0,0,0,0,0,0,1,0,0,0
1,855801065,6.4134,1.2319,7.0522,1.2319,7.0421,1.3435,6.4185,1.3435,101,...,0,0,0,0,0,0,0,1,0,0
2,® Langer Kornweg 34K 65451 - Kelsterbach Germa...,1.3891,0.4411,3.0267,0.4411,3.0267,0.9176,1.3891,0.9176,101,...,0,0,0,0,0,0,0,0,1,0
3,Willy-Rüsch-Str. 4-10 71349 KERNEN,0.8883,2.2147,2.2033,2.2181,2.2021,2.6909,0.8871,2.6875,101,...,0,0,0,0,0,0,1,0,0,0
4,80066508,7.0558,2.0709,7.7411,2.0709,7.7411,2.2029,7.0558,2.2029,101,...,0,0,0,0,0,0,0,1,0,0


In [282]:
## last time data preparation

x = df_final.copy()
y = df_final[labels]
for i in range(len(labels)):
    del x[labels[i]]
del x['value']

In [283]:
## data split for training and testing/evaluating

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [296]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

OneVsRest_model = OneVsRestClassifier(
    # LogisticRegression(C=1.8, solver="lbfgs", penalty="l2", max_iter=250)
    XGBClassifier(eval_metric='mlogloss')
)
OneVsRest_model.fit(x_train, y_train)
predictions = OneVsRest_model.predict(x_test)

print("the accuracy on test dataset", accuracy_score(y_test, predictions))

/home/woody/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
the accuracy on test dataset 0.7692307692307693
/home/woody/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [311]:
## some thoughts for future work:
# 1) there might be some link between different features, we can train a simple NN to solve the multicollinearity issue
# 2) more detailed parameter tuning for XGBoost and also try other algorithms, I tried SVM, LR and decision tree. We can try more models
# 3) Maybe we can try stacking models to improve the accuracy